## Intro

When Librosa loads an audio file, there is a default sample rate, which is the number of samples from the mp3's audio signal for one second. Therefore,
to get the length of a song in real time we take the length of the read-in array and divide by the sample rate for that audio sample.

In [ ]:
# system packages
import os
import warnings

# usual imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# tensorflow and keras
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.models import Sequential

## audio tools
import librosa
from torchaudio import info

# my functions and classes
from utilities import (
    view_melspec,
    read_metadata_file,
    Batch_generator,
    id_from_path,
    attach_onehot_encoding,
    mp3_to_mel_path
)

# for timing loops
import tqdm

warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
# VERY GLOBAL VARIABLES
audio_dir = "./data/fma_small/"
global_dur = 0.5
global_sr = 8000

The next code chunk is Super handy function that just searches for every mp3 file within the audio directory. 
My audio folder has a bunch of subfolders like '000', '099', etc, and the audio files are
one level further down.

In [ ]:
filepaths = librosa.util.files.find_files(audio_dir)

Here is what a typical load looks like. Specifying `sr=None` ensures that the file's default sample-rate is preserved. 
We can also specify the duration with `librosa.load`.

In [ ]:
example_source, sr = librosa.load(filepaths[1], sr=None, duration=2)
librosa.get_duration(example_source, sr=sr)

In [ ]:
plt.plot(example_source)
plt.title(
    f"The first {librosa.get_duration(example_source, sr=sr)} seconds of an audio file"
)
plt.xlabel("Sample Position")
plt.ylabel("Amplitude")
plt.show()

In [ ]:
view_melspec(example_source, sr)

## How long does a typical load take?

On my linux virtual machine, one second of audio takes between 143 and 214 milliseconds to load one second of audio (down to 129 ms if we don't force a sample rate). 
Multiplying this number by 8 is about how long it will take to load the entire dataset in. I estimate about 15 minutes for the whole FMA Small dataset.

However, on my macbook it takes about 16 ms to load one second of audio in. For five seconds of audio, about 34 ms. This is much faster than the ubuntu machine! This means an estimated 134 seconds, or about two minutes.

This runs a lot faster at 270 microseconds on my ubuntu virtual machine, so it will be a better tool to 
check for integrity. On the macbook it takes about 330 microseconds, a bit slower.


## Check for integrity

This is the only point where I will use `torchaudio` for the `torchaudio.info` method. This method
tries to open the file without loading it into memory. This allows us to check
for corrupted mp3 files. If this leads to a very long list of files, double check the integrity
of your download.

In [ ]:
bad_files = []
too_short_duration = 5.0  # seconds
for file in filepaths:
    try:
        info_obj = info(file)[0]

        # Add a file to the bad list if it is shorter than 5 seconds.
        if info_obj.length / (info_obj.rate * info_obj.channels) < 5.0:
            bad_files.append(file)

    except RuntimeError:
        bad_files.append(file)
bad_files

## Estimating memory cost

## Managing Metadata 

The following code takes the giant metadata file coming with the FMA datasets and selects
the track ids and the genres.

In [ ]:
metadata_path = os.path.join("data", "fma_metadata", "tracks.csv")

# Read in the metadata and process it. See utilities.py for explanation.
meta_df = read_metadata_file(metadata_path, filepaths, bad_files)

# shuffle the dataset
meta_df = meta_df.sample(frac=1, random_state=1)

# get a list of the genres
genre_list = list(meta_df.genre.unique())

In [ ]:
## check to make sure each path points to the right file
meta_df["track_id"].equals(meta_df["mp3_path"].apply(lambda x: int(id_from_path(x))))

In [ ]:
meta_df.head()

## Preprocessing

In [ ]:
melspec_dir = os.path.join(".", "data", "fma_small_melspecs")

The next cell will process `.mp3` files into melspectrograms, which are numpy arrays. Numpy arrays are saved as `.npz` files. The next cell will (most likely) be set to a raw cell, so that it does not run when executing the entire notebook.

(Turning a cell from code to raw is like an on-off switch for a cell!)

In [ ]:
meta_df['mel_path'] = meta_df['mp3_path'].apply(lambda x: mp3_to_mel_path(x, melspec_dir))

Almost done. Need to troubleshoot why the glob is failing to find the corresponding .npz file.

## Loading Data

The class `Batch_generator` loads in batches one at a time so as to not load a huge amount of data into the kernel.

In [ ]:
batch_size = 8

training_generator = Batch_generator(
    meta_df.iloc[:4800, :], batch_size=batch_size, sr=global_sr, duration=global_dur
)
validation_generator = Batch_generator(
    meta_df.iloc[4800:6400, :], batch_size=batch_size, sr=global_sr, duration=global_dur
)
test_generator = Batch_generator(
    meta_df.iloc[6400:, :], batch_size=batch_size, sr=global_sr, duration=global_dur
)

## Logistic Regression

It's worth it to see if we can train a logistic regression model and see how well it does at classifying the genres. 

In [ ]:
input_shape = training_generator[0][0].shape[1:]

## CNN

First we explore how well a traditional convolutional neural net works on this dataset.